# Word embedding models

In [3]:
from koselleck import *

In [4]:
get_neighbors??

Signature: get_neighbors(w, m1, m2=None, topn=25, wide=True)
Docstring: <no docstring>
Source:   
def get_neighbors(w,m1,m2=None,topn=25,wide=True):
    ms=[m1,m2] if m2 is not None else [m1]
    df=pd.DataFrame([
        {
            'word1':w,
            'word2':w2,
            'model':mi,
            'csim':csim,
            'crank':wi
        }
        for mi,m in enumerate(ms)
        for wi,(w2,csim) in enumerate(m.wv.most_similar(w,topn=topn))
    ])
    if not wide:
        return df
    else:
        odf=None
        for i,gdf in sorted(df.groupby('model')):
            gdf=gdf.set_index('crank').sort_index()
            if odf is None: odf=pd.DataFrame(index=gdf.index)
            odf[f'model_{i}']=gdf.word2
        return odf
File:      ~/github/koselleck/koselleck/embeddings.py
Type:      function


## Cache existing data in db

In [2]:
save_vecs_in_db(num_proc=4)

opening Sqlite table 'vecs' in '/home/ryan/github/koselleck/data/db/db.koselleck.vecs.sqlite'
Mapping do_save_vecs_in_db() [x4]: 100%|██████████| 480/480 [01:44<00:00,  4.61it/s]


In [2]:
save_vecs_in_db(num_proc=4)

Mapping do_save_vecs_in_db() [x4]: 100%|██████████| 480/480 [01:47<00:00,  4.48it/s]


In [ ]:
vecs()

In [5]:
vl=get_veclib('vecs')

In [9]:
# list(vl.keys())

In [ ]:
stop

In [17]:
vecs('1720-1725',2)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
the,0.473669,-0.106946,0.157871,0.064637,-0.003224,0.656484,-0.105191,0.112247,1.487899,0.010959,...,0.455802,-0.480213,1.241615,-0.572774,-0.326102,0.343117,-0.877381,0.278451,-0.992694,-1.611975
of,-0.022396,0.088112,-0.120756,0.601107,0.319101,-1.807469,-0.429939,1.053778,0.033382,-0.791587,...,-0.111624,-0.644188,0.732212,-0.583835,-0.842624,1.188061,-0.231490,1.103414,-0.483656,-1.469839
to,1.792318,0.554737,-0.739299,0.143966,0.024411,1.497402,-1.329036,-1.202589,1.221623,0.785355,...,-0.956781,-0.908250,1.239368,-0.731752,-1.470767,0.815075,-0.810960,-0.446697,1.278480,-1.667818
and,1.015325,-0.656740,0.109203,-0.312618,0.504945,0.101595,-0.676754,0.695306,0.895752,0.256065,...,-0.074896,0.028524,1.421799,-0.337816,-0.631439,-0.020605,0.324697,0.152265,-0.159660,-1.029040
in,-0.844093,-0.517567,-0.083109,0.436985,0.565679,0.297170,0.412478,1.055781,-0.486433,0.486129,...,-0.950282,-0.606701,1.267293,-0.386480,-0.234282,0.977757,0.183932,2.416256,-1.333853,-0.703754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
lwp,0.004429,-0.088704,-0.072314,-0.006049,-0.106655,-0.210816,0.042529,0.176739,0.068044,-0.050715,...,0.072268,0.091187,0.153558,-0.029034,0.037162,-0.009522,0.093188,0.018297,0.056745,-0.234281
scut,0.087382,0.189057,0.199472,0.253323,0.065417,-0.077732,0.157637,0.328398,-0.008661,0.112979,...,-0.122499,0.069879,-0.017118,-0.134644,0.254794,-0.017945,-0.030729,-0.008236,-0.124368,-0.393111
eras,-0.052435,0.234638,0.004772,0.163921,0.071496,-0.452706,0.168584,0.434802,-0.270504,-0.300793,...,0.286194,0.128699,0.272772,-0.134437,0.332998,0.091798,0.030149,-0.168987,-0.104180,-0.153753
heiau,-0.097873,0.086641,-0.028302,-0.103358,0.055717,-0.108400,0.077790,0.109701,-0.047668,-0.103743,...,0.012839,-0.026876,0.100193,-0.151117,0.146897,0.121632,-0.033672,-0.028547,-0.082556,0.029754


In [8]:
dbget('vec(virtue_1720-1725)')

In [ ]:
stop

## Generate data

### Create models

In [ ]:
def get_skipgrams(idir=PATH_SKIPGRAMS_YR,skipgram_n=25, calc_numlines=False):
    odf=pd.DataFrame([
        {
            'corpus':fn.split('.')[2],
            'year':int([x for x in fn.split('.') if x.isdigit()][0]),
#             'period_end':int([x for x in fn.split('.') if x.isdigit()][-1]),
            'path':os.path.join(idir,fn)
        }
        for fn in os.listdir(idir)
        if fn.startswith('data.skipgrams')
    ]).sort_values(['corpus','year'])
    if calc_numlines:
        odf['num_lines']=odf.path.progress_apply(get_numlines)
        odf['num_words']=odf['num_lines']*skipgram_n
    return odf#.query('1680<=year<1970')

In [ ]:
# get_skipgrams()

In [ ]:
dfskip=get_skipgrams(calc_numlines=True)
# dfskip=dfskip.query('1720<=year<1900')
dfskip['period']=dfskip.year.apply(lambda y: periodize(y,YEARBIN))
dfskip

In [ ]:
# dfskip.groupby('period').num_words.sum()

In [ ]:
def get_dfskipruns(dfskip,num_runs=10,incl_existing=False):
    dfskipruns=pd.concat([
        dfskip.assign(run=f'run_{str(i+1).zfill(2)}')
        for i in range(num_runs)
    ])
    dfskipruns['opath']=dfskipruns.apply(lambda row: os.path.join(PATH_MODELS_NEW,row.corpus,row.period,row.run,'model.bin'),1)
    dfskipruns['opath_exists']=dfskipruns.opath.apply(lambda x: os.path.exists(x))
    if not incl_existing: dfskipruns=dfskipruns[dfskipruns.opath_exists==False]
    return dfskipruns

In [ ]:
# get_dfskipruns(dfskip,num_runs=2)

In [ ]:
def gen_and_save_model(dfskip,nskip=DEFAULT_NUM_SKIP,force=False,vector_size=100,window=10,min_count=5,epochs=10,workers=8,verbose=False):
    row=dfskip.iloc[0]
    odir=os.path.join(PATH_MODELS_NEW,row.corpus,row.period,row.run)
    ofnfn=os.path.join(odir,'model.bin')
    if force or not os.path.exists(ofnfn):
        ensure_dir_exists(odir)
        ss=SkipgramsSamplers(dfskip.path, nskip)
        disable_gensim_logging() if not verbose else enable_gensim_logging()
        model = Word2Vec(sentences=ss,vector_size=vector_size,window=window,min_count=min_count,epochs=epochs,workers=workers)
        model.save(ofnfn)
    return pd.DataFrame([{'fnfn':ofnfn}])


In [ ]:
# fnfn=gen_and_save_model(get_dfskipruns(dfskip).iloc[:1], force=True).fnfn.iloc[0]
# load_model(fnfn).wv.most_similar('value')

In [ ]:
# res=pmap_groups(
#     gen_and_save_model,
#     dfskipruns.groupby(['period','run']),
#     num_proc=4,
#     kwargs=dict(force=True, nskip=NSKIP_PER_YR)
# )

In [ ]:
def gen_models(
        ybin=5,
        ymin=1680,
        ymax=1970,
        num_runs=1,
        force=False,
        nskip_per_yr=NSKIP_PER_YR
    ):
    dfskip=get_skipgrams(calc_numlines=False).query(f'{ymin}<=year<{ymax}')
    dfskip['period']=dfskip.year.apply(lambda y: periodize(y,ybin))
    dfskipruns=get_dfskipruns(dfskip, num_runs=num_runs, incl_existing=force)
    dfgrps=dfskipruns.groupby(['period','run'])
    print(f'Generating {len(dfgrps)} new models over {dfskipruns.period.nunique()} periods and {dfskipruns.run.nunique()} runs')
    return pmap_groups(
        gen_and_save_model,
        dfskipruns.groupby(['period','run']),
        num_proc=4,
        kwargs=dict(force=force, nskip=nskip_per_yr)
    )

In [ ]:
gen_models(num_runs=10)

## Test models

In [ ]:
m=load_model('/home/ryan/github/koselleck/data/models/bpo/1805-1810/run_25/model.bin')
m.wv.most_similar('virtue')

In [ ]:
m=load_model('/home/ryan/github/koselleck/data/models/bpo/1965-1970/run_07/model.bin')
m.wv.most_similar(['king','woman'],['man'])

In [ ]:
def test_models(dfmodels,gby=['period','run']):
    o=[]
    dfgrp=dfmodels.groupby(gby)
    for period,dfg in tqdm(sorted(dfgrp)):#, total=len(dfgrp)):
        path=dfg.iloc[-1].path
        m=load_model(path)
        try:
            testvec=m.wv.most_similar(['king','woman'],['man'],topn=25)
        except KeyError:
            continue
        testvec_wl=[x for x,y in testvec]
        has_queen='queen' in set(testvec_wl)
        odx={
            **dict(zip(gby,period)),
            'has_queen':has_queen,
            'rank_queen':testvec_wl.index('queen') if has_queen else np.nan,
            'neighborhood':', '.join(testvec_wl),
        }
        o+=[odx]
#         break
    return pd.DataFrame(o)

In [ ]:
dfmodels = get_pathdf_models().query('period_len==5')
dftests  = test_models(dfmodels)
dftests

In [ ]:
dftests.to_csv('../../data/data.model.tests.csv')

In [ ]:
dftests.query('has_queen==True').groupby('period').size()